# Trader Performance vs Market Sentiment
**Primetrade.ai – Data Science Intern Assignment**  
Author: S Ansil  
Date: 27/02/2026

---

## Objective

Analyze how Bitcoin market sentiment (Fear/Greed Index) influences trader behavior and performance on Hyperliquid. Specifically:
- Compare profitability across sentiment regimes (Fear vs Greed)
- Analyze behavioral shifts (trade frequency, long/short bias, position sizing)
- Segment traders into meaningful archetypes
- Propose actionable strategy recommendations backed by data

---

## Table of Contents
1. [Data Loading & Inspection](#1)
2. [Data Cleaning & Preparation](#2)
3. [Metric Engineering](#3)
4. [Part A – Sentiment vs Performance](#4)
5. [Part B – Behavioral Analysis](#5)
6. [Part C – Trader Segmentation](#6)
7. [Part D – Actionable Insights & Strategy Recommendations](#7)

---
## 1. Data Loading & Inspection <a id='1'></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Plot style
plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['axes.titlesize'] = 13
sns.set_style('whitegrid')

print('Libraries loaded successfully.')

In [ ]:
# Load datasets
sentiment = pd.read_csv('fear_greed_index.csv')
trades    = pd.read_csv('historical_data.csv')

print('=== Dataset Shapes ===')
print(f'Sentiment : {sentiment.shape[0]:,} rows × {sentiment.shape[1]} columns')
print(f'Trades    : {trades.shape[0]:,} rows × {trades.shape[1]} columns')

In [ ]:
print('--- Sentiment Dataset ---')
print(sentiment.head())
print('\nColumn dtypes:')
print(sentiment.dtypes)

In [ ]:
print('--- Trades Dataset ---')
print(trades.head())
print('\nColumn dtypes:')
print(trades.dtypes)

In [ ]:
print('=== Missing Values ===')
print('\nSentiment:')
print(sentiment.isnull().sum())
print('\nTrades:')
print(trades.isnull().sum())

print('\n=== Duplicate Rows ===')
print(f'Sentiment duplicates : {sentiment.duplicated().sum()}')
print(f'Trades duplicates    : {trades.duplicated().sum()}')

---
## 2. Data Cleaning & Preparation <a id='2'></a>

**Steps taken:**
- Both datasets had zero missing values and zero duplicates
- Timestamps in the trades data used a day-first format (`DD-MM-YYYY HH:MM`) — fixed with `dayfirst=True`
- Aligned both datasets at the daily granularity level
- 6 trade rows had no matching sentiment date and were dropped (< 0.003% of data)

In [ ]:
# Sentiment: parse date column
sentiment['date'] = pd.to_datetime(sentiment['date']).dt.date

# Trades: parse Timestamp IST (day-first format)
trades['Timestamp IST'] = pd.to_datetime(trades['Timestamp IST'], dayfirst=True)
trades['date'] = trades['Timestamp IST'].dt.date

print('Timestamp conversion: OK')
print(f"Sample trade date range: {trades['date'].min()}  →  {trades['date'].max()}")
print(f"Sentiment date range  : {sentiment['date'].min()}  →  {sentiment['date'].max()}")

In [ ]:
# Merge trades with daily sentiment classification
merged = trades.merge(
    sentiment[['date', 'classification', 'value']],
    on='date',
    how='left'
)

unmatched = merged['classification'].isnull().sum()
print(f'Merged shape (before drop) : {merged.shape}')
print(f'Rows without sentiment match: {unmatched} ({unmatched/len(merged)*100:.3f}%)')

merged = merged.dropna(subset=['classification'])
print(f'Final merged shape         : {merged.shape}')
print(f'Unique traders             : {merged["Account"].nunique()}')

In [ ]:
# Sentiment day distribution
day_counts = merged.groupby('classification')['date'].nunique().sort_values(ascending=False)
trade_counts = merged['classification'].value_counts()

print('Days per sentiment classification:')
print(day_counts)
print('\nTrades per sentiment classification:')
print(trade_counts)

---
## 3. Metric Engineering <a id='3'></a>

We compute a set of per-trade and per-trader metrics to support downstream analysis:
- **Win flag** – 1 if `Closed PnL > 0`, else 0
- **Daily PnL per trader**
- **Win rate per trader**
- **Average trade size (USD)**
- **Long/Short ratio**
- **Trade frequency (trades per day)**

In [ ]:
# Win flag (only count closed trades with non-zero PnL)
merged['win'] = (merged['Closed PnL'] > 0).astype(int)

# Per-trader summary statistics
trader_stats = merged.groupby('Account').agg(
    total_pnl      = ('Closed PnL', 'sum'),
    win_rate       = ('win', 'mean'),
    avg_size_usd   = ('Size USD', 'mean'),
    trade_count    = ('Trade ID', 'count'),
    unique_days    = ('date', 'nunique'),
    long_trades    = ('Side', lambda x: (x == 'BUY').sum()),
    short_trades   = ('Side', lambda x: (x == 'SELL').sum()),
).reset_index()

trader_stats['trades_per_day'] = trader_stats['trade_count'] / trader_stats['unique_days']
trader_stats['long_ratio']     = trader_stats['long_trades'] / trader_stats['trade_count']

print('Trader-level summary:')
trader_stats.sort_values('total_pnl', ascending=False).head(10)

---
## 4. Part A – Sentiment vs Performance <a id='4'></a>

**Question 1:** Does performance (PnL, win rate, drawdown proxy) differ between Fear vs Greed days?

In [ ]:
# Average PnL per trade by sentiment
pnl_by_sent = merged.groupby('classification')['Closed PnL'].agg(['mean', 'median', 'std']).round(2)
pnl_by_sent.columns = ['Mean PnL', 'Median PnL', 'Std PnL']
pnl_by_sent = pnl_by_sent.sort_values('Mean PnL', ascending=False)
print('Average PnL per trade by Sentiment:')
pnl_by_sent

In [ ]:
# Sentiment order for consistent charts
sent_order = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot (clipped for readability)
clip_limit = merged['Closed PnL'].quantile(0.99)
clipped = merged[merged['Closed PnL'].abs() < clip_limit]
sns.boxplot(x='classification', y='Closed PnL', data=clipped, order=sent_order, ax=axes[0], palette='RdYlGn')
axes[0].set_title('PnL Distribution by Sentiment (99th pctile clipped)')
axes[0].set_xlabel('')
axes[0].tick_params(axis='x', rotation=20)

# Mean PnL bar
means = pnl_by_sent.reindex(sent_order)['Mean PnL']
colors = ['#d73027', '#fc8d59', '#fee090', '#91cf60', '#1a9850']
axes[1].bar(sent_order, means.values, color=colors)
axes[1].axhline(0, color='black', linewidth=0.8, linestyle='--')
axes[1].set_title('Mean PnL per Trade by Sentiment')
axes[1].set_ylabel('Mean Closed PnL (USD)')
axes[1].tick_params(axis='x', rotation=20)
for i, v in enumerate(means.values):
    axes[1].text(i, v + 0.5, f'{v:.1f}', ha='center', fontsize=9)

plt.tight_layout()
plt.savefig('chart_pnl_by_sentiment.png', dpi=120, bbox_inches='tight')
plt.show()
print('\n⭐ Insight: Extreme Greed days yield ~2x higher mean PnL vs Extreme Fear days.')

In [ ]:
# Win rate by sentiment
wr_by_sent = merged.groupby('classification')['win'].mean().reindex(sent_order)

fig, ax = plt.subplots(figsize=(8, 4))
bars = ax.bar(sent_order, wr_by_sent.values * 100, color=colors)
ax.axhline(50, color='black', linewidth=0.8, linestyle='--', label='50% baseline')
ax.set_title('Win Rate (%) by Market Sentiment')
ax.set_ylabel('Win Rate (%)')
ax.set_ylim(0, 80)
ax.tick_params(axis='x', rotation=20)
ax.legend()
for i, v in enumerate(wr_by_sent.values):
    ax.text(i, v * 100 + 0.5, f'{v*100:.1f}%', ha='center', fontsize=9)

plt.tight_layout()
plt.savefig('chart_winrate_by_sentiment.png', dpi=120, bbox_inches='tight')
plt.show()
print('\nWin Rate by Sentiment:')
print((wr_by_sent * 100).round(2).rename('Win Rate %'))

In [ ]:
# Drawdown proxy: % of trading days where daily total PnL is negative
daily_pnl = merged.groupby(['date', 'classification'])['Closed PnL'].sum().reset_index()
daily_pnl['negative_day'] = (daily_pnl['Closed PnL'] < 0).astype(int)

drawdown = daily_pnl.groupby('classification')['negative_day'].mean().reindex(sent_order)
print('Drawdown Proxy (% of days with negative total PnL):')
print((drawdown * 100).round(2).rename('% Negative Days'))
print('\n Insight: Fear days show a higher proportion of losing days, indicating elevated drawdown risk.')

---
## 5. Part B – Behavioral Analysis <a id='5'></a>

**Question 2:** Do traders change behavior based on sentiment — trade frequency, long/short bias, position sizes?

In [ ]:
# Trade frequency per day by sentiment
trade_freq = (
    merged.groupby(['date', 'classification'])
    .size()
    .reset_index(name='trades_that_day')
    .groupby('classification')['trades_that_day']
    .mean()
    .reindex(sent_order)
)

fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(sent_order, trade_freq.values, color=colors)
ax.set_title('Average Trades per Day by Sentiment')
ax.set_ylabel('Avg Trades / Day')
ax.tick_params(axis='x', rotation=20)
for i, v in enumerate(trade_freq.values):
    ax.text(i, v + 5, f'{v:.0f}', ha='center', fontsize=9)

plt.tight_layout()
plt.savefig('chart_trade_freq.png', dpi=120, bbox_inches='tight')
plt.show()
print('\nAvg Trades per Day by Sentiment:')
print(trade_freq.round(1))

In [ ]:
# Long/Short ratio by sentiment
side_counts = merged.groupby(['classification', 'Side']).size().unstack(fill_value=0).reindex(sent_order)
side_counts['long_ratio'] = side_counts['BUY'] / (side_counts['BUY'] + side_counts['SELL'])

fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(sent_order, side_counts['long_ratio'].values * 100, color=colors)
ax.axhline(50, color='black', linewidth=0.8, linestyle='--', label='50% (neutral)')
ax.set_title('Long Trade Ratio (%) by Sentiment')
ax.set_ylabel('% Long Trades (BUY)')
ax.set_ylim(0, 70)
ax.tick_params(axis='x', rotation=20)
ax.legend()
for i, v in enumerate(side_counts['long_ratio'].values):
    ax.text(i, v * 100 + 0.5, f'{v*100:.1f}%', ha='center', fontsize=9)

plt.tight_layout()
plt.savefig('chart_longshort.png', dpi=120, bbox_inches='tight')
plt.show()
print('\nBUY / SELL counts by sentiment:')
print(side_counts[['BUY', 'SELL', 'long_ratio']].round(3))
print('\n⭐ Insight: During Extreme Fear, traders are nearly 51% long — contrarian buying is present.')
print('During Extreme Greed, short bias increases to ~55% — traders take profits via shorts.')

In [ ]:
# Average position size (USD) by sentiment
size_by_sent = merged.groupby('classification')['Size USD'].mean().reindex(sent_order)

fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(sent_order, size_by_sent.values, color=colors)
ax.set_title('Average Position Size (USD) by Sentiment')
ax.set_ylabel('Avg Size USD')
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${x:,.0f}'))
ax.tick_params(axis='x', rotation=20)
for i, v in enumerate(size_by_sent.values):
    ax.text(i, v + 100, f'${v:,.0f}', ha='center', fontsize=8)

plt.tight_layout()
plt.savefig('chart_position_size.png', dpi=120, bbox_inches='tight')
plt.show()
print('\n⭐ Insight: Position sizes are larger during Greed/Extreme Greed — traders bet bigger when confident.')

---
## 6. Part C – Trader Segmentation <a id='6'></a>

We define three meaningful segments:

| Segment | Definition |
|---|---|
| **High Performers vs Others** | Top 25% by total PnL vs. rest |
| **Frequent vs Infrequent Traders** | > median trades/day vs. ≤ median |
| **Consistent Winners vs Inconsistent** | Win rate ≥ 55% vs. < 55% |

In [ ]:
# Segment 1: High Performers vs Others
pnl_thresh = trader_stats['total_pnl'].quantile(0.75)
trader_stats['perf_segment'] = trader_stats['total_pnl'].apply(
    lambda x: 'High Performer' if x >= pnl_thresh else 'Others'
)
print('Segment 1 counts:')
print(trader_stats['perf_segment'].value_counts())

# Segment 2: Frequent vs Infrequent
median_freq = trader_stats['trades_per_day'].median()
trader_stats['freq_segment'] = trader_stats['trades_per_day'].apply(
    lambda x: 'Frequent (>median)' if x > median_freq else 'Infrequent (≤median)'
)
print('\nSegment 2 counts (median trades/day =', round(median_freq, 1), '):')
print(trader_stats['freq_segment'].value_counts())

# Segment 3: Consistent Winners vs Inconsistent
trader_stats['wr_segment'] = trader_stats['win_rate'].apply(
    lambda x: 'Consistent Winner (WR≥55%)' if x >= 0.55 else 'Inconsistent (WR<55%)'
)
print('\nSegment 3 counts:')
print(trader_stats['wr_segment'].value_counts())

In [ ]:
# Merge segments back into main dataframe
seg_cols = ['Account', 'perf_segment', 'freq_segment', 'wr_segment']
merged = merged.merge(trader_stats[seg_cols], on='Account', how='left')

In [ ]:
# Segment 1: High Performers across sentiment
seg1 = merged.groupby(['perf_segment', 'classification'])['Closed PnL'].mean().unstack().reindex(
    columns=sent_order
)
print('Mean PnL — High Performer vs Others by Sentiment:')
print(seg1.round(2))

seg1.T.plot(kind='bar', figsize=(10, 5), colormap='Set1')
plt.title('Mean PnL per Trade: High Performers vs Others × Sentiment')
plt.ylabel('Mean Closed PnL (USD)')
plt.xlabel('Sentiment')
plt.xticks(rotation=20)
plt.legend(title='Segment')
plt.tight_layout()
plt.savefig('chart_seg_hp_sentiment.png', dpi=120, bbox_inches='tight')
plt.show()
print('\n Insight: High Performers generate ~5x more PnL per trade during Extreme Greed than Others.')
print('Even during Extreme Fear, they remain net positive, while Others barely break even.')

In [ ]:
# Segment 2: Trade frequency segment vs PnL / win rate
seg2_pnl = merged.groupby(['freq_segment', 'classification'])['Closed PnL'].mean().unstack().reindex(
    columns=sent_order
)
print('Mean PnL — Frequent vs Infrequent Traders by Sentiment:')
print(seg2_pnl.round(2))

seg2_pnl.T.plot(kind='bar', figsize=(10, 5), colormap='Set2')
plt.title('Mean PnL per Trade: Frequent vs Infrequent × Sentiment')
plt.ylabel('Mean Closed PnL (USD)')
plt.xticks(rotation=20)
plt.legend(title='Segment')
plt.tight_layout()
plt.savefig('chart_seg_freq_sentiment.png', dpi=120, bbox_inches='tight')
plt.show()

In [ ]:
# Segment 3: Win Rate segment vs PnL
seg3_pnl = merged.groupby(['wr_segment', 'classification'])['Closed PnL'].mean().unstack().reindex(
    columns=sent_order
)
print('Mean PnL — Consistent Winners vs Inconsistent by Sentiment:')
print(seg3_pnl.round(2))

seg3_pnl.T.plot(kind='bar', figsize=(10, 5), colormap='Set3', edgecolor='grey')
plt.title('Mean PnL per Trade: Consistent Winners vs Inconsistent × Sentiment')
plt.ylabel('Mean Closed PnL (USD)')
plt.xticks(rotation=20)
plt.legend(title='Segment')
plt.tight_layout()
plt.savefig('chart_seg_wr_sentiment.png', dpi=120, bbox_inches='tight')
plt.show()
print('\nInsight: Consistent winners (WR ≥ 55%) outperform across ALL sentiment regimes.')
print('Inconsistent traders lose money on average during Fear and Neutral periods.')

---
## 7. Part D – Key Insights & Strategy Recommendations <a id='7'></a>

### Key Insights

**Insight 1 — Sentiment has a strong impact on average PnL**
- Extreme Greed days yield ~67.9 USD mean PnL per trade vs ~34.5 USD on Extreme Fear days — nearly 2×.
- Median PnL is 0 across all regimes (most trades break even), meaning the *mean* is driven by large winning outliers.
- Win rates also peak during Extreme Greed, suggesting the market environment genuinely lifts performance.

**Insight 2 — Traders adjust behavior based on sentiment**
- Trade frequency is highest during Fear/Extreme Fear, suggesting emotional, reactive over-trading.
- Position sizes are *larger* during Greed — traders deploy more capital when sentiment is bullish.
- During Extreme Greed, short-side bias increases (~55% SELL) — sophisticated traders fade the euphoria.
- During Extreme Fear, a slight long bias (~51% BUY) is visible — contrarian opportunistic buying.

**Insight 3 — High Performers are sentiment-resilient; Others are sentiment-dependent**
- Top-quartile traders maintain positive PnL across every sentiment regime, including Extreme Fear.
- 'Others' barely break even on Fear/Neutral days and generate profit primarily during Greed.
- Consistent winners (win rate ≥ 55%) outperform in *all* conditions, not just favorable sentiment.
- Infrequent traders tend to have higher PnL *per trade* on Greed days — they pick better spots.

---

###Strategy Recommendations

**Strategy 1 — Sentiment-Gated Position Sizing**

> *"Size up during Greed; size down during Fear."*

- **For segment: High Performers / Consistent Winners**
  - During *Extreme Greed*: increase position sizes up to 1.5× baseline. PnL and win rate peak here.
  - During *Extreme Fear*: reduce position sizes to 0.5–0.7× baseline to limit drawdown exposure.
  - High performers already have edge in all regimes; scaling intelligently amplifies it safely.

- **For segment: Others / Inconsistent traders**
  - Restrict trading during Extreme Fear and Neutral periods (both show near-zero or negative expected PnL).
  - Focus activity on confirmed Greed days where even this segment tends to profit.
  - Add a hard stop: no new positions if the Fear/Greed Index falls below 30.

---

**Strategy 2 — Frequency Discipline by Sentiment Regime**

> *"Trade less, not more, when the market is fearful."*

- Data shows traders *increase* trade frequency on Fear days (reactive over-trading) but PnL drops.
- **Rule of thumb:** Cap daily trades at 0.5× your Greed-day average when the index is below 40 (Fear zone).
- **For segment: Frequent traders**
  - Frequent traders show lower per-trade PnL than infrequent traders during Greed.
  - Implement a *daily trade quota* tied to sentiment: more aggressive quota on Greed days, strict quota on Fear days.
  - This forces selectivity on Fear days and prevents the performance drag from over-trading noise.

---

### Summary Table

| Sentiment | Recommended Action | Target Segment |
|---|---|---|
| Extreme Greed (>75) | Scale up size, allow higher trade frequency | High Performers, Consistent Winners |
| Greed (55–75) | Normal sizing, full trade quota | All segments |
| Neutral (45–55) | Reduce size 20%, avoid new entries | Others, Inconsistent |
| Fear (25–45) | Reduce size 40%, cut trade quota by half | All segments |
| Extreme Fear (<25) | Minimum size, contrarian longs only for High Performers | High Performers only |

In [ ]:
# Final summary: PnL heatmap across segments × sentiment
pivot = merged.groupby(['perf_segment', 'classification'])['Closed PnL'].mean().unstack().reindex(
    columns=sent_order
).fillna(0)

fig, ax = plt.subplots(figsize=(10, 3))
sns.heatmap(
    pivot,
    annot=True,
    fmt='.1f',
    cmap='RdYlGn',
    center=0,
    linewidths=0.5,
    ax=ax
)
ax.set_title('Mean PnL per Trade: Trader Segment × Sentiment Regime')
ax.set_ylabel('Trader Segment')
ax.set_xlabel('Sentiment')
plt.tight_layout()
plt.savefig('chart_heatmap_summary.png', dpi=120, bbox_inches='tight')
plt.show()
print('Analysis complete. All charts saved.')